# W1M3 - ETL 프로세스 구현하기

In [202]:
import requests
from bs4 import BeautifulSoup

In [203]:
html = requests.get("https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29").text

In [204]:
soup = BeautifulSoup(html, 'html.parser')

In [205]:
gdp = soup.find("table", {"class": 'wikitable'}).find('tbody').find_all('tr')[3:]

In [206]:
gdp_imf = [[gdp_row.find_all('td')[i].text.strip() for i in range(2)] for gdp_row in gdp]
gdp_imf

[['United States', '30,337,162'],
 ['China', '19,534,894'],
 ['Germany', '4,921,563'],
 ['Japan', '4,389,326'],
 ['India', '4,271,922'],
 ['United Kingdom', '3,730,261'],
 ['France', '3,283,429'],
 ['Italy', '2,459,597'],
 ['Canada', '2,330,308'],
 ['Brazil', '2,307,162'],
 ['Russia', '2,195,708'],
 ['South Korea', '1,947,133'],
 ['Australia', '1,881,140'],
 ['Spain', '1,827,576'],
 ['Mexico', '1,817,818'],
 ['Indonesia', '1,492,618'],
 ['Turkey', '1,455,413'],
 ['Netherlands', '1,272,960'],
 ['Saudi Arabia', '1,136,580'],
 ['Switzerland', '999,604'],
 ['Poland', '915,451'],
 ['Taiwan', '814,438'],
 ['Belgium', '689,364'],
 ['Sweden', '638,780'],
 ['Ireland', '587,225'],
 ['Argentina', '574,201'],
 ['United Arab Emirates', '568,567'],
 ['Singapore', '561,725'],
 ['Austria', '559,218'],
 ['Israel', '550,905'],
 ['Thailand', '545,341'],
 ['Norway', '503,466'],
 ['Malaysia', '488,250'],
 ['Philippines', '471,516'],
 ['Vietnam', '468,400'],
 ['Iran', '463,747'],
 ['Bangladesh', '451,856'],

In [249]:
import pandas as pd

df_gdp = pd.DataFrame(gdp_imf)
df_gdp

,0,1
0,United States,"30,337,162"
1,China,"19,534,894"
2,Germany,"4,921,563"
3,Japan,"4,389,326"
4,India,"4,271,922"
...,...,...
204,Kiribati,311
205,Palau,308
206,Marshall Islands,305
207,Nauru,161


In [250]:
df_gdp.to_json('raw_data_gdp.json')

In [251]:
pd.read_json('raw_data_gdp.json')

,0,1
0,United States,"30,337,162"
1,China,"19,534,894"
2,Germany,"4,921,563"
3,Japan,"4,389,326"
4,India,"4,271,922"
...,...,...
204,Kiribati,311
205,Palau,308
206,Marshall Islands,305
207,Nauru,161


In [252]:
df_gdp.columns = ['country', 'gdp']
df_gdp

,country,gdp
0,United States,"30,337,162"
1,China,"19,534,894"
2,Germany,"4,921,563"
3,Japan,"4,389,326"
4,India,"4,271,922"
...,...,...
204,Kiribati,311
205,Palau,308
206,Marshall Islands,305
207,Nauru,161


In [253]:
df_gdp['gdp'] = df_gdp['gdp'].apply(lambda x : int(x.replace(",","")) if x != '—' else None)
df_gdp['gdp'] = round(df_gdp['gdp']/1000, 2)
df_gdp

,country,gdp
0,United States,30337.16
1,China,19534.89
2,Germany,4921.56
3,Japan,4389.33
4,India,4271.92
...,...,...
204,Kiribati,0.31
205,Palau,0.31
206,Marshall Islands,0.30
207,Nauru,0.16


In [254]:
df_gdp = df_gdp.sort_values('gdp', ascending = False)
df_gdp

,country,gdp
0,United States,30337.16
1,China,19534.89
2,Germany,4921.56
3,Japan,4389.33
4,India,4271.92
...,...,...
178,Greenland,NaN
184,Eritrea,NaN
185,Zanzibar,NaN
192,Sint Maarten,NaN


In [255]:
df_100B = df_gdp[df_gdp['gdp'] >= 100]
df_100B

,country,gdp
0,United States,30337.16
1,China,19534.89
2,Germany,4921.56
3,Japan,4389.33
4,India,4271.92
...,...,...
68,Uzbekistan,112.65
69,Guatemala,112.37
70,Oman,109.99
71,Bulgaria,108.42


In [294]:
import pycountry
import pycountry_convert as pc

alpha2 = pycountry.countries.search_fuzzy("Türkiye")[0].alpha_2
con_code = pc.country_alpha2_to_continent_code(alpha2)
con_code

'AS'

In [ ]:
alpha2 = pycountry.countries.search_fuzzy("Türkiye")[0].alpha_2
con_code = pc.country_alpha2_to_continent_code(alpha2)
con_code

In [327]:
region_map = {'DR Congo' : 'Democratic Republic of the Congo', 'Turkey' : 'Türkiye', 'kosovo' : 'Kosovo'}

In [328]:
df_gdp['country'] = df_gdp['country'].map(region_map).fillna(df_gdp['country'])
df_gdp['country']

0                 United States
1                         China
2                       Germany
3                         Japan
4                         India
                 ...           
178                   Greenland
184                     Eritrea
185                    Zanzibar
192                Sint Maarten
195    Turks and Caicos Islands
Name: country, Length: 209, dtype: object

In [332]:
df_gdp[df_gdp['country'] == 'Kosovo']

,country,gdp
151,Kosovo,11.32


In [333]:
df_gdp['continent'] = df_gdp['country'].apply(lambda x : pc.country_alpha2_to_continent_code(pc.country_name_to_country_alpha2(x)))
df_gdp

KeyError: "Invalid Country Name: 'Kosovo'"

In [ ]:
df_gdp['continent'] = df_gdp['country'].apply(lambda x : pc.country_alpha2_to_continent_code(pc.countries.search_fuzzy(x)[0].alpha_2))
df_gdp

In [338]:
def country_to_continent(x) :
        try : continent = pc.country_alpha2_to_continent_code(pc.country_name_to_country_alpha2(x))
        except :
            exception_region_dict = {'DR Congo' : 'AF', 'Kosovo' : 'EU', 'Sint Maarten' : 'EU', 'Zanzibar' : 'AF', 'East Timor' : 'AS'}
            continent = exception_region_dict[x]
        return continent

df_gdp['continent'] = df_gdp['country'].apply(lambda x : country_to_continent(x))

In [339]:
df_gdp.groupby('continent').head(5).groupby('continent').mean('gdp').sort_values('gdp', ascending = False)

,gdp
continent,
NA,6946.500
AS,6327.178
EU,3318.112
SA,791.566
OC,436.658
AF,285.184


In [89]:
import sqlite3

conn = sqlite3.connect('World_Economies.db')
df_100B.to_sql('Countries_by_GDP', conn, index = False)

72

In [ ]:
cur = conn.cursor()

In [90]:
pd.read_sql(
    """
    SELECT * FROM COUNTRIES_BY_GDP""", conn)

,country,gdp,continent
0,United States,30337.16,NA
1,China,19534.89,AS
2,Germany,4921.56,EU
3,Japan,4389.33,AS
4,India,4271.92,AS
...,...,...,...
67,Uzbekistan,112.65,AS
68,Guatemala,112.37,NA
69,Oman,109.99,AS
70,Bulgaria,108.42,EU


In [98]:
def exceed_100B() :
    conn = sqlite3.connect('World_Economies.db')
    cur = conn.cursor()
    df = pd.read_sql("SELECT * FROM Countries_by_GDP WHERE GDP > 100", conn)
    return df

In [179]:
def region_top5_average() :
    conn = sqlite3.connect('World_Economies.db')
    cur = conn.cursor()
    df = pd.read_sql(
        """SELECT AVG(GDP), CONTINENT
        FROM (SELECT * FROM 
        (SELECT *, ROW_NUMBER() OVER (PARTITION BY CONTINENT ORDER BY GDP DESC) AS RANK 
        FROM Countries_by_GDP) 
        WHERE RANK <=5)
        GROUP BY CONTINENT
        """, conn)
    return df

In [180]:
exceed_100B()

,country,gdp,continent
0,United States,30337.16,NA
1,China,19534.89,AS
2,Germany,4921.56,EU
3,Japan,4389.33,AS
4,India,4271.92,AS
...,...,...,...
67,Uzbekistan,112.65,AS
68,Guatemala,112.37,NA
69,Oman,109.99,AS
70,Bulgaria,108.42,EU


In [181]:
region_top5_average()

,AVG(GDP),continent
0,285.184,AF
1,6327.178,AS
2,3318.112,EU
3,6946.500,NA
4,1072.030,OC
5,791.566,SA


In [165]:
pd.read_sql(
        """SELECT *
        FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY CONTINENT ORDER BY GDP DESC) AS RANK 
        FROM Countries_by_GDP)
        WHERE RANK <= 5
        """, conn)

,country,gdp,continent,RANK
0,South Africa,418.05,AF,1
1,Egypt,380.04,AF,2
2,Algeria,264.27,AF,3
3,Nigeria,194.96,AF,4
4,Morocco,168.60,AF,5
5,China,19534.89,AS,1
6,Japan,4389.33,AS,2
7,India,4271.92,AS,3
8,South Korea,1947.13,AS,4
9,Indonesia,1492.62,AS,5


In [178]:
pd.read_sql(
        """SELECT AVG(GDP), CONTINENT
        FROM (SELECT * FROM 
        (SELECT *, ROW_NUMBER() OVER (PARTITION BY CONTINENT ORDER BY GDP DESC) AS RANK 
        FROM Countries_by_GDP) 
        WHERE RANK <=5)
        GROUP BY CONTINENT
        """, conn)

,AVG(GDP),continent
0,285.184,AF
1,6327.178,AS
2,3318.112,EU
3,6946.500,NA
4,1072.030,OC
5,791.566,SA


In [342]:
import etl_project_gdp as epg
epg.ETL_gdp()

          country       gdp
0   United States  30337.16
1           China  19534.89
2         Germany   4921.56
3           Japan   4389.33
4           India   4271.92
..            ...       ...
68     Uzbekistan    112.65
69      Guatemala    112.37
70           Oman    109.99
71       Bulgaria    108.42
72      Venezuela    106.33

[72 rows x 2 columns]
                gdp
continent          
NA         6946.500
AS         6327.178
EU         3318.112
SA          791.566
OC          436.658
AF          285.184
